In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Pobieranie i czyszczenie danych
data = pd.read_excel("clinical_radiomics_imported_from_tsv2.xlsx")
data_cleaned = data.dropna()
data_cleaned = data.copy()  # Tworzenie kopii danych

# Obliczenie mediany kolumny OS
median_OS = data_cleaned["OS"].median()

# Dodanie etykiet
data_cleaned["label"] = np.where(data_cleaned["OS"] <= median_OS, 0, 1)

# Przygotowanie danych
columns_original = list(data_cleaned.loc[:, "original_shape_Elongation":"original_ngtdm_Strength"])
X = data_cleaned[columns_original]
y = data_cleaned["label"]

# Podział danych na zbiór treningowy i walidacyjny
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
# Inicjalizacja modelu Random Forest
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

# Skalowanie cech
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Perform feature selection using RFE
rfe = RFE(estimator=rf, n_features_to_select=15)
X_train_rfe = rfe.fit_transform(X_train_scaled, y_train)
X_val_rfe = X_val[X_train.columns[rfe.support_]]

# Fit the SVM model on the selected features
svm = SVC()
svm.fit(X_train_rfe, y_train)

# Predict on the validation set
X_val_scaled = scaler.transform(X_val)
X_val_rfe = rfe.transform(X_val_scaled)
y_val_pred = svm.predict(X_val_rfe)
accuracy = accuracy_score(y_val, y_val_pred)
print("Accuracy of SVM after feature selection: {:.2f}%".format(accuracy * 100))


Accuracy of SVM after feature selection: 73.33%


In [ ]:
print

In [2]:
# Define the parameter grid for GridSearchCV
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(svm, param_grid, cv=5)
grid_search.fit(X_train_rfe, y_train)

# Get the best hyperparameters and retrain the SVM model
best_params = grid_search.best_params_
svm = SVC(**best_params)
svm.fit(X_train_rfe, y_train)
# Predict on the validation set
X_val_scaled = scaler.transform(X_val)
X_val_rfe = rfe.transform(X_val_scaled)
y_val_pred = svm.predict(X_val_rfe)
accuracy = accuracy_score(y_val, y_val_pred)
print("Accuracy of SVM after feature selection: {:.2f}%".format(accuracy * 100))

Accuracy of SVM after feature selection: 73.33%


In [3]:
# Define the parameter grid for RandomForestClassifier
rf_param_grid = {
    'n_estimators': [100, 200, 500],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10]
}

# Perform grid search for RandomForestClassifier
rf_grid_search = GridSearchCV(rf, rf_param_grid, cv=5)
rf_grid_search.fit(X_train, y_train)

# Get the best RandomForestClassifier model
best_rf_model = rf_grid_search.best_estimator_

# Define the parameter grid for SVC
svm_param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

# Perform grid search for SVC
svm_grid_search = GridSearchCV(svm, svm_param_grid, cv=5)
svm_grid_search.fit(X_train_rfe, y_train)

# Get the best SVC model
#best_svm_model = svm_grid_search.best_estimator_
# Predict on the validation set
X_val_scaled = scaler.transform(X_val)
X_val_rfe = rfe.transform(X_val_scaled)
y_val_pred = svm.predict(X_val_rfe)
accuracy = accuracy_score(y_val, y_val_pred)
print("Accuracy of SVM after feature selection: {:.2f}%".format(accuracy * 100))

Accuracy of SVM after feature selection: 73.33%


In [4]:
# Wybór optymalnych cech
feature_importance = rf.feature_importances_
sorted_indices = np.argsort(feature_importance)[::-1]
selected_features = X.columns[sorted_indices[:15]]
# Aktualizacja danych treningowych i walidacyjnych
X_train_selected = X_train[selected_features]
X_val_selected = X_val[selected_features]
 
svm.fit(X_train_selected, y_train)
# Ocena modelu na danych walidacyjnych
y_val_pred = svm.predict(X_val_selected)
accuracy = accuracy_score(y_val, y_val_pred)
print("Dokładność klasyfikatora SVM po optymalizacji: {:.2f}%".format(accuracy * 100))

Dokładność klasyfikatora SVM po optymalizacji: 66.67%


In [5]:
print(selected_features)

Index(['original_firstorder_RootMeanSquared',
       'original_firstorder_10Percentile',
       'original_glrlm_RunLengthNonUniformityNormalized',
       'original_glrlm_RunEntropy', 'original_glrlm_RunPercentage',
       'original_glszm_HighGrayLevelZoneEmphasis',
       'original_firstorder_Maximum', 'original_glcm_SumEntropy',
       'original_firstorder_InterquartileRange', 'original_firstorder_Mean',
       'original_firstorder_Minimum', 'original_glcm_DifferenceAverage',
       'original_shape_Elongation', 'original_glcm_Contrast',
       'original_glcm_Idmn'],
      dtype='object')


In [6]:
# Define the parameter grid for GridSearchCV
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(svm, param_grid, cv=5)
grid_search.fit(X_train_rfe, y_train)

# Get the best hyperparameters and retrain the SVM model
best_params = grid_search.best_params_
svm = SVC(**best_params)
svm.fit(X_train_selected, y_train)
# Ocena modelu na danych walidacyjnych
y_val_pred = svm.predict(X_val_selected)
accuracy = accuracy_score(y_val, y_val_pred)
print("Dokładność klasyfikatora SVM po optymalizacji: {:.2f}%".format(accuracy * 100))

Dokładność klasyfikatora SVM po optymalizacji: 66.67%


In [7]:
# Define the parameter grid for RandomForestClassifier
rf_param_grid = {
    'n_estimators': [100, 200, 500],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10]
}

# Perform grid search for RandomForestClassifier
rf_grid_search = GridSearchCV(rf, rf_param_grid, cv=5)
rf_grid_search.fit(X_train, y_train)

# Get the best RandomForestClassifier model
best_rf_model = rf_grid_search.best_estimator_

# Define the parameter grid for SVC
svm_param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

# Perform grid search for SVC
svm_grid_search = GridSearchCV(svm, svm_param_grid, cv=5)
svm_grid_search.fit(X_val_selected, y_train)

#svm.fit(X_train_selected, y_train)
# Ocena modelu na danych walidacyjnych
y_val_pred = svm.predict(X_val_selected)
accuracy = accuracy_score(y_val, y_val_pred)
print("Dokładność klasyfikatora SVM po optymalizacji: {:.2f}%".format(accuracy * 100))

ValueError: Found input variables with inconsistent numbers of samples: [15, 59]